In [1]:
import os, glob

import pandas as pd
import numpy as np

from dask.distributed import Client

import dask.dataframe as dd

import altair as alt

In [2]:
## Key code for read CSV files into pandas and Dask
def Build_DTypes_Dict(filename_list):
    df = pd.read_csv(filename_list[0], nrows=1) # read the first row of the first file and get column names
    
    dtype = dict(zip(sorted(df.columns), ['object'] * len(df.columns)))
    
    # ! important - may need to change other fields to category, int, date etc for better memory management
    dtype['federal_action_obligation'] = 'float64'
    
    # read date fields as datetime
    # convert many others to category type
    
    #dtype['action_date_fiscal_year'] = 'int'
    
    return dtype

In [3]:
def Collect_Parquet_Files_List():
    path_dir = "USAspending_Archive_Downloads/Expanded_CSV_Files"
    usa_parquet_files = sorted(glob.glob(os.path.join(path_dir, "FY*_CSV.parquet")))
    return usa_parquet_files

def Collect_CSV_Files_List():
    path_dir = "USAspending_Archive_Downloads/Expanded_CSV_Files"
    usa_parquet_files = sorted(glob.glob(os.path.join(path_dir, "FY*.csv")))
    return usa_parquet_files

In [4]:
usa_parquet_files = Collect_Parquet_Files_List()
usa_parquet_files

['USAspending_Archive_Downloads/Expanded_CSV_Files/FY2010_All_Contracts_Full_20200807_1_CSV.parquet',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2010_All_Contracts_Full_20200807_2_CSV.parquet',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2010_All_Contracts_Full_20200807_3_CSV.parquet',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2010_All_Contracts_Full_20200807_4_CSV.parquet',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2011_All_Contracts_Full_20200807_1_CSV.parquet',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2011_All_Contracts_Full_20200807_2_CSV.parquet',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2011_All_Contracts_Full_20200807_3_CSV.parquet',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2011_All_Contracts_Full_20200807_4_CSV.parquet',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2012_All_Contracts_Full_20200807_1_CSV.parquet',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2012_All_Contracts_Full_20200

In [5]:
usa_csv_files = Collect_CSV_Files_List()
usa_csv_files

['USAspending_Archive_Downloads/Expanded_CSV_Files/FY2010_All_Contracts_Full_20200807_1.csv',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2010_All_Contracts_Full_20200807_2.csv',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2010_All_Contracts_Full_20200807_3.csv',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2010_All_Contracts_Full_20200807_4.csv',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2011_All_Contracts_Full_20200807_1.csv',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2011_All_Contracts_Full_20200807_2.csv',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2011_All_Contracts_Full_20200807_3.csv',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2011_All_Contracts_Full_20200807_4.csv',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2012_All_Contracts_Full_20200807_1.csv',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2012_All_Contracts_Full_20200807_2.csv',
 'USAspending_Archive_Downloads/Expanded_CSV_Files/FY2012_Al

In [6]:
#usa_parquet_files[0]

In [7]:
# %%time
# df = pd.read_parquet(usa_parquet_files[0])

# df['action_date_fiscal_year'].unique()

# df['federal_action_obligation'].sum()

# df.memory_usage().sum()/1e9

In [13]:
if 'client' in locals() or 'client' in globals(): client.close()
    
client = Client()

client

Client Scheduler: tcp://127.0.0.1:51804 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 68.72 GB


In [29]:
### Read DASK DF from CSV Files

usa_csv_file_list = usa_csv_files = Collect_CSV_Files_List()
usa_csv_file_list

dtypes = Build_DTypes_Dict(usa_csv_file_list)
dtypes

ddf = dd.read_csv(usa_csv_file_list, dtype = dtypes)
ddf = ddf.fillna('UNSPECIFIED')
ddf['PSC_Cat'] = ddf['product_or_service_code'].str[:1]
ddf['PSC_Cat2'] = ddf['product_or_service_code'].str[:2]

In [31]:
#ddf.visualize()

In [32]:
# %%time
# ddf.groupby(['action_date_fiscal_year'])['federal_action_obligation'].sum().compute()

In [33]:
# for field in ddf.columns:
#     print(field, ddf[field].dtype)

#### ALTERNATE - Read from Collection of Smaller Parquet Files into Dask (faster operations - preferred for frequent DASK requests)

In [34]:
#usa_parquet_files = Collect_Parquet_Files_List()

#ddf = dd.read_parquet(usa_parquet_files) #, usecols=['action_date_fiscal_year', 'federal_action_obligation'])

In [35]:
%%time
ddf.head()

CPU times: user 81.2 ms, sys: 12.2 ms, total: 93.4 ms
Wall time: 2.06 s


,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,...,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,last_modified_date,PSC_Cat,PSC_Cat2
0,9700_9700_0006_3_W91WAW09D0003_0,CONT_AWD_0006_9700_W91WAW09D0003_9700,0006,3,0,9700,DEPT OF DEFENSE,W91WAW09D0003,0,0.00,...,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,https://www.usaspending.gov/#/award/CONT_AWD_0...,2020-08-01 13:31:00,R,R4
1,4730_-NONE-_GS28F0001V_33_-NONE-_-NONE-,CONT_IDV_GS28F0001V_4730,GS28F0001V,33,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,0.00,...,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,https://www.usaspending.gov/#/award/CONT_IDV_G...,2016-08-26 00:00:00,7,71
2,8000_-NONE-_GSFC0200004DNAS500136_55_-NONE-_0,CONT_AWD_GSFC0200004DNAS500136_8000_-NONE-_-NONE-,GSFC0200004DNAS500136,55,0,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,840000.00,...,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,https://www.usaspending.gov/#/award/CONT_AWD_G...,2016-08-24 00:00:00,A,AR
3,9700_9700_0013_2_W91WAW09D0003_0,CONT_AWD_0013_9700_W91WAW09D0003_9700,0013,2,0,9700,DEPT OF DEFENSE,W91WAW09D0003,0,186174.39,...,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,https://www.usaspending.gov/#/award/CONT_AWD_0...,2020-08-01 13:26:29,R,R4
4,9700_-NONE-_N0010410PFH86_P00001_-NONE-_0,CONT_AWD_N0010410PFH86_9700_-NONE-_-NONE-,N0010410PFH86,P00001,0,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,0,0.00,...,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,UNSPECIFIED,https://www.usaspending.gov/#/award/CONT_AWD_N...,2010-10-13 00:00:00,4,48


### DASK Compute

In [284]:
def Fix_Parent_Names_Large_Firms_Inplace(df):  # more fixes are needed here
    
    # df_share_by_PSC[df_share_by_PSC['recipient_parent_name'].str.startswith('LEIDOS ')]['recipient_parent_name'].value_counts()
    # after GFY2014, ['SAIC  INC.', 'SAIC GEMINI  INC.', 'SAIC SOLUTIONS LTD'] should be 'LEIDOS HOLDINGS, INC.' or ['LEIDOS HOLDINGS, INC.', 'LEIDOS HOLDINGS  INC.', 'LEIDOS  INC.','LEIDOS BIOMEDICAL RESEARCH  INC.','LEIDOS INTERMEDIATE HOLDINGS  INC.']
    
    legacy_saic_inc_variations = df_share_by_PSC[df_share_by_PSC['recipient_parent_name'].str.startswith('SAIC ')]['recipient_parent_name'].unique()
    
    selector = (df['recipient_parent_name'].isin(legacy_saic_inc_variations)) & (~df['action_date_fiscal_year'].isin(['2010','2011','2012','2014',]))
    df.loc[selector,'recipient_parent_name'] = 'LEIDOS'
    
    # repeat selector since df values have now changed
    selector = (df['recipient_parent_name'].isin(legacy_saic_inc_variations)) & (~df['action_date_fiscal_year'].isin(['2010','2011','2012','2014',]))
    assert list(df[selector]['recipient_parent_name'].unique()) == [] # confirm it changed those
    
    leidos_variations = df_share_by_PSC[df_share_by_PSC['recipient_parent_name'].str.startswith('LEIDOS')]['recipient_parent_name'].unique()
    selector = (df['recipient_parent_name'].isin(leidos_variations))
    df.loc[selector,'recipient_parent_name'] = 'LEIDOS'

    return

In [270]:
%%time
#12/5/2020

selector_group = ['action_date_fiscal_year', 'funding_agency_name', 'funding_sub_agency_name','funding_office_name','PSC_Cat','product_or_service_code','product_or_service_code_description','recipient_parent_name']
df_by_Dept_Agency_Office_PSC_Contractor = ddf.groupby(selector_group)['federal_action_obligation'].sum()



df_by_Dept_Agency_Office_PSC_Contractor = df_by_Dept_Agency_Office_PSC_Contractor.compute() #launched DASK processes to gather the data

df_by_Dept_Agency_Office_PSC_Contractor = df_by_Dept_Agency_Office_PSC_Contractor.reset_index()

In [285]:
#!!! Key Fixes
df_by_Dept_Agency_Office_PSC_Contractor = df_by_Dept_Agency_Office_PSC_Contractor.fillna("UNSPECIFIED")
Fix_Parent_Names_Large_Firms_Inplace(df_by_Dept_Agency_Office_PSC_Contractor)
df_by_Dept_Agency_Office_PSC_Contractor['PSC_Cat'] = df_by_Dept_Agency_Office_PSC_Contractor['product_or_service_code'].str[:1]
df_by_Dept_Agency_Office_PSC_Contractor['PSC_Cat2'] = df_by_Dept_Agency_Office_PSC_Contractor['product_or_service_code'].str[:2]

df_by_Dept_Agency_Office_PSC_Contractor.shape

(8185097, 10)

In [287]:
df_by_Dept_Agency_Office_PSC_Contractor.memory_usage().sum()/1e9

0.654807888

In [288]:
df_by_Dept_Agency_Office_PSC_Contractor.head()

,action_date_fiscal_year,funding_agency_name,funding_sub_agency_name,funding_office_name,PSC_Cat,product_or_service_code,product_or_service_code_description,recipient_parent_name,federal_action_obligation,PSC_Cat2
0,2010,ACCESS BOARD (USAB),ARCHITECTURAL AND TRANSPORTATION BARRIERS COMP...,ARCHITECTURAL AND TRANSPORTATION BARRIERS COMP...,3,3610,"PRINTING,DUPLICATING & BOOKBIND EQ",CANON INC.,10005.72,36
1,2010,ACCESS BOARD (USAB),ARCHITECTURAL AND TRANSPORTATION BARRIERS COMP...,ARCHITECTURAL AND TRANSPORTATION BARRIERS COMP...,J,J058,MAINT-REP OF COMMUNICATION EQ,NETCO INC.,9826.96,J0
2,2010,ACCESS BOARD (USAB),ARCHITECTURAL AND TRANSPORTATION BARRIERS COMP...,ARCHITECTURAL AND TRANSPORTATION BARRIERS COMP...,R,R606,COURT REPORTING SERVICES,METRO REPORTERS INC,6375.00,R6
3,2010,ACCESS BOARD (USAB),ARCHITECTURAL AND TRANSPORTATION BARRIERS COMP...,UNSPECIFIED,S,S113,TELEPHONE AND-OR COMMUNICATIONS SER,VERIZON COMMUNICATIONS INC.,-136.93,S1
4,2010,ADVISORY COUNCIL ON HISTORIC PRESERVATION (ACHP),ADVISORY COUNCIL ON HISTORIC PRESERVATION,ADVISORY COUNCIL ON HISTORIC PRESERVATION,7,7035,ADP SUPPORT EQUIPMENT,NITROSECURITY INC.,40785.00,70


In [563]:
#!!!!TEST
def GFY_Obligation_Totals_Checksum_TEST(df1, df2):
    """This is an important checksum to confirm that all obligation $s are accounted for after a groupby transformation"""
    df_1_total_obs = df1.groupby(['action_date_fiscal_year'])['federal_action_obligation'].sum().reset_index()
    df_2_total_obs = df2.groupby(['action_date_fiscal_year'])['federal_action_obligation'].sum().reset_index()
    df_test = df_1_total_obs.merge(df_2_total_obs, on=['action_date_fiscal_year'])
    difference = sum(df_test['federal_action_obligation_x'] - df_test['federal_action_obligation_y'])
    if difference < 1.0:
        return True
    else:
        return False

In [289]:
df_by_Dept_Agency_Office_PSC_Contractor.to_parquet("GFY10-GFY20YTD_df_by_Dept_Agency_Office_PSC_Contractor.parquet")

In [290]:
df_obligations_gfy = df_by_Dept_Agency_Office_PSC_Contractor.groupby(['action_date_fiscal_year'])['federal_action_obligation'].sum().reset_index()
df_obligations_gfy 

,action_date_fiscal_year,federal_action_obligation
0,2010,5.610369e+11
1,2011,5.398079e+11
2,2012,5.208065e+11
3,2013,4.633853e+11
4,2014,4.461897e+11
5,2015,4.397271e+11
6,2016,4.754231e+11
7,2017,5.105917e+11
8,2018,5.553379e+11
9,2019,5.898982e+11


In [291]:
df_share_by_PSC = df_by_Dept_Agency_Office_PSC_Contractor.groupby(['action_date_fiscal_year', 'PSC_Cat','recipient_parent_name'])['federal_action_obligation'].sum().reset_index()
assert GFY_Obligation_Totals_Checksum_TEST(df_by_Dept_Agency_Office_PSC_Contractor, df_share_by_PSC)
df_share_by_PSC.head()

,action_date_fiscal_year,PSC_Cat,recipient_parent_name,federal_action_obligation
0,2010,1,126536 CANADA INC,70080.0
1,2010,1,17TH ST. PHOTO SUPPLY INC.,11564.9
2,2010,1,1DB - METRAVIB,27970.0
3,2010,1,200 PARK INC.,20250.0
4,2010,1,2037706 ONTARIO INC,11909.0


#### Get Largest - Test

In [801]:
selector = (df_share_by_PSC['action_date_fiscal_year'] == '2019') & (df_share_by_PSC['PSC_Cat'] == 'D') & (df_share_by_PSC['federal_action_obligation'] > 0.0)
df_share_by_PSC.iloc[df_share_by_PSC[selector]['federal_action_obligation'].rank().sort_values(ascending=False).index].head(10)

,action_date_fiscal_year,PSC_Cat,recipient_parent_name,federal_action_obligation
2328474,2019,D,GENERAL DYNAMICS CORPORATION,2.945345e+09
2329981,2019,D,LEIDOS,2.255022e+09
2331470,2019,D,PERSPECTA INC.,1.789726e+09
2326076,2019,D,BOOZ ALLEN HAMILTON HOLDING CORPORATION,1.592884e+09
2324938,2019,D,ACCENTURE INC.,1.179230e+09
2332363,2019,D,SCIENCE APPLICATIONS INTERNATIONAL CORPORATION,9.079429e+08
2334068,2019,D,VERIZON COMMUNICATIONS INC.,8.146354e+08
2326267,2019,D,CACI INTERNATIONAL INC,8.028504e+08
2331033,2019,D,NORTHROP GRUMMAN CORPORATION,7.822535e+08
2326472,2019,D,CERNER GOVERNMENT SERVICES INC.,7.232623e+08


In [295]:
#df_share_by_PSC.pivot_table(values=['federal_action_obligation'], columns=['action_date_fiscal_year'], index=['PSC_Cat', 'recipient_parent_name'], fill_value=0.0, aggfunc=np.sum)

### Save Intermediate Results in a Parquet File for Ease of Recovery

In [296]:
df_share_by_PSC.to_parquet("GFY10-20YTD_df_share_by_PSC.parquet")

#### Construct a new tidy table 

In [297]:
# totals by GFY, PSC_Cat
df_share_by_PSC_totals = df_share_by_PSC.groupby(['action_date_fiscal_year', 'PSC_Cat'])['federal_action_obligation'].sum().reset_index()
df_share_by_PSC_totals.rename(columns={'federal_action_obligation' : 'Obligations_Total_All_Firms'}, inplace=True)
df_share_by_PSC_totals = df_share_by_PSC_totals[['action_date_fiscal_year', 'PSC_Cat', 'Obligations_Total_All_Firms']]
df_share_by_PSC_totals

,action_date_fiscal_year,PSC_Cat,Obligations_Total_All_Firms
0,2010,1,6.688913e+10
1,2010,2,2.917819e+10
2,2010,3,2.854948e+09
3,2010,4,5.577365e+09
4,2010,5,2.389621e+10
...,...,...,...
358,2020,V,9.062542e+09
359,2020,W,8.734846e+08
360,2020,X,3.068328e+08
361,2020,Y,2.313687e+10


In [298]:
# confirm that the totals add up to GFY sum totals above - check to make sure nothing is lost in translation
sum(df_share_by_PSC_totals.groupby(['action_date_fiscal_year'])['Obligations_Total_All_Firms'].sum().to_list()) - sum(df_obligations_gfy['federal_action_obligation'].to_list())

-0.0439453125

In [299]:
df_share_by_PSC.head()

,action_date_fiscal_year,PSC_Cat,recipient_parent_name,federal_action_obligation
0,2010,1,126536 CANADA INC,70080.0
1,2010,1,17TH ST. PHOTO SUPPLY INC.,11564.9
2,2010,1,1DB - METRAVIB,27970.0
3,2010,1,200 PARK INC.,20250.0
4,2010,1,2037706 ONTARIO INC,11909.0


In [525]:
# helper function to get nth item (index 0 is first) sorted values by obligations

def Get_Nth_Company_By_Obligation_Amount(df, nth_value=0): # 0 is the first
    return df.sort_values(by=['action_date_fiscal_year', 'PSC_Cat','federal_action_obligation'], ascending=[True, True, False]).groupby(['action_date_fiscal_year', 'PSC_Cat']).nth(nth_value).reset_index()

Get_Nth_Company_By_Obligation_Amount(df_share_by_PSC,0)

,action_date_fiscal_year,PSC_Cat,recipient_parent_name,federal_action_obligation
0,2010,1,LOCKHEED MARTIN CORPORATION,1.329083e+10
1,2010,2,OSHKOSH CORPORATION,6.802084e+09
2,2010,3,CATERPILLAR INC.,2.554212e+08
3,2010,4,BECHTEL GROUP INC.,1.702873e+09
4,2010,5,LOCKHEED MARTIN CORPORATION,2.059365e+09
...,...,...,...,...
358,2020,V,UNITED LAUNCH ALLIANCE L.L.C.,9.406926e+08
359,2020,W,DEPLOYED RESOURCES LLC,9.391296e+07
360,2020,X,NIH BAYVIEW ACQUISITION LLC,4.824487e+07
361,2020,Y,FISHER SAND & GRAVEL CO.,2.141814e+09


### Begin Analysis of Top N Largest Companies - Market Share Calculations

In [664]:
# Company #1, #2, #5, #10 totals

#df_share_by_PSC.sort_values(by=['action_date_fiscal_year', 'PSC_Cat','federal_action_obligation'], ascending=[True, True, False]).groupby(['action_date_fiscal_year', 'PSC_Cat']).nth(0)

## Add the Top_N to df_share_by_PSC_totals

#df_share_by_PSC_totals.head()

def Compute_Sum_N_largest_Parent_Recipients(df, group_by_list, N_list=[1,2,5]):
#     df_result = group['federal_action_obligation'].apply(lambda xgrp: xgrp.nlargest(N).sum()).reset_index()
#     df_result.rename(columns={"federal_action_obligation" : f"Top_{N}_Companies_Obligations"}, inplace=True)

    # slow operation -should save if multiple repeats
    df_grouped_sorted_by_recipient = df.groupby(group_by_list + ['recipient_parent_name'])['federal_action_obligation'].sum().reset_index().sort_values(by=['action_date_fiscal_year', 'PSC_Cat','federal_action_obligation'], ascending=[True, True, False])
    
    # do first item to create df_result
    N = N_list[0]
    df_result = df_grouped_sorted_by_recipient.groupby(group_by_list)['federal_action_obligation'].apply(lambda x: x.nlargest(N).sum()).reset_index()
    df_result.rename(columns={"federal_action_obligation" : f"Top_{N}_Companies_Obligations"}, inplace=True)
    
    for N in N_list[1:]: # loop and merge the rest
        df_X = df_grouped_sorted_by_recipient.groupby(group_by_list)['federal_action_obligation'].apply(lambda x: x.nlargest(N).sum()).reset_index()
        df_X.rename(columns={"federal_action_obligation" : f"Top_{N}_Companies_Obligations"}, inplace=True)
        df_result = df_result.merge(df_X, on=group_by_list)
    
    
    return df_result

#TEST Compute_Sum_N_largest_Parent_Recipients(df_by_Dept_Agency_Office_PSC_Contractor, group_by_list=['action_date_fiscal_year','PSC_Cat'], N_list=[1,2,5])

In [665]:
def Create_df_combined(df, by_group=['PSC_Cat']):
    
    #groupby and get nlargest -  https://stackoverflow.com/questions/40390634/pandas-groupby-nlargest-sum
    by_full_group = ['action_date_fiscal_year'] + by_group
    grp = df.groupby(by_full_group)
    
    df_share_by_PSC_totals = grp['federal_action_obligation'].sum().reset_index()
    
    df_N_Largest = Compute_Sum_N_largest_Parent_Recipients(df, group_by_list=by_full_group, N_list=[1,2,5])
    df_combined = df_share_by_PSC_totals.merge(df_N_Largest, on=by_full_group)
    
    df_combined['action_date_fiscal_year_numeric'] = df_combined['action_date_fiscal_year'].astype(int)
    cols_display = ['action_date_fiscal_year','action_date_fiscal_year_numeric'] + by_group + ['federal_action_obligation',
                     'Top_1_Companies_Obligations','Top_2_Companies_Obligations', 'Top_5_Companies_Obligations']
    df_combined = df_combined[cols_display]

    df_combined['Top_1_Marketshare'] = df_combined['Top_1_Companies_Obligations']/df_combined['federal_action_obligation']
    df_combined['Top_2_Marketshare'] = df_combined['Top_2_Companies_Obligations']/df_combined['federal_action_obligation']
    df_combined['Top_5_Marketshare'] = df_combined['Top_5_Companies_Obligations']/df_combined['federal_action_obligation']
    
    return df_combined.fillna(0.0)

In [672]:
%%time
df_combined= Create_df_combined(df_by_Dept_Agency_Office_PSC_Contractor, by_group=['PSC_Cat'])
#TEST CHECK
assert GFY_Obligation_Totals_Checksum_TEST(df_by_Dept_Agency_Office_PSC_Contractor, df_combined)
df_combined

CPU times: user 9.22 s, sys: 635 ms, total: 9.86 s
Wall time: 9.55 s


,action_date_fiscal_year,action_date_fiscal_year_numeric,PSC_Cat,federal_action_obligation,Top_1_Companies_Obligations,Top_2_Companies_Obligations,Top_5_Companies_Obligations,Top_1_Marketshare,Top_2_Marketshare,Top_5_Marketshare
0,2010,2010,1,6.688913e+10,1.329083e+10,2.463369e+10,4.130159e+10,0.198699,0.368276,0.617463
1,2010,2010,2,2.917819e+10,6.802084e+09,9.336014e+09,1.550904e+10,0.233122,0.319965,0.531528
2,2010,2010,3,2.854948e+09,2.554212e+08,4.354778e+08,8.260521e+08,0.089466,0.152534,0.289341
3,2010,2010,4,5.577365e+09,1.702873e+09,2.335968e+09,2.821523e+09,0.305319,0.418830,0.505888
4,2010,2010,5,2.389621e+10,2.059365e+09,4.023967e+09,8.936834e+09,0.086180,0.168393,0.373985
...,...,...,...,...,...,...,...,...,...,...
358,2020,2020,V,9.062542e+09,9.406926e+08,1.633101e+09,3.361216e+09,0.103800,0.180203,0.370891
359,2020,2020,W,8.734846e+08,9.391296e+07,1.421079e+08,1.991655e+08,0.107515,0.162691,0.228013
360,2020,2020,X,3.068328e+08,4.824487e+07,6.728485e+07,1.118728e+08,0.157235,0.219288,0.364605
361,2020,2020,Y,2.313687e+10,2.141814e+09,3.368273e+09,5.810549e+09,0.092571,0.145580,0.251138


In [673]:
#df_combined[df_combined['PSC_Cat']=='D']

#### <move to top> Get Product Or Service Code Master List from Acquisition.GOV

In [674]:
df_product_or_service_code_lookup = df_by_Dept_Agency_Office_PSC_Contractor[['product_or_service_code', 'product_or_service_code_description']].drop_duplicates() #.compute()
df_product_or_service_code_lookup.head()

,product_or_service_code,product_or_service_code_description
0,3610,"PRINTING,DUPLICATING & BOOKBIND EQ"
1,J058,MAINT-REP OF COMMUNICATION EQ
2,R606,COURT REPORTING SERVICES
3,S113,TELEPHONE AND-OR COMMUNICATIONS SER
4,7035,ADP SUPPORT EQUIPMENT


In [675]:
def Retrieve_PSC_Main_Name():
    # from the master source at acquisition.gov
    df_psc_codes_master = pd.read_excel('https://www.acquisition.gov/sites/default/files/manual/PSC_Data_October_2020.xlsx', sheet_name="PSC Data")
    #df_psc_codes_master.head()
    result_lookup_dict = {}
    df_psc_codes_master_CURRENT = df_psc_codes_master[df_psc_codes_master['END DATE'].isnull()].copy().reset_index() # have no End Date means they are current 
    df_psc_codes_master_CURRENT['PSC CODE STRING'] = df_psc_codes_master_CURRENT['PSC CODE'].apply(str)
    single_letter_definitions = df_psc_codes_master_CURRENT[df_psc_codes_master_CURRENT['PSC CODE STRING'].apply(len) == 1][['PSC CODE STRING', 'PRODUCT AND SERVICE CODE NAME']]
    
    selector = (df_psc_codes_master_CURRENT['PSC CODE STRING'].apply(len) == 2) & (df_psc_codes_master_CURRENT['PSC CODE STRING'].str[:1].apply(lambda x: x.isnumeric()))
    two_number_definitions = df_psc_codes_master_CURRENT[selector][['PSC CODE STRING', 'PRODUCT AND SERVICE CODE NAME']]
    
    two_number_definitions_ending_in_0 = two_number_definitions[two_number_definitions['PSC CODE STRING'].str[1:2]=='0']
    
    two_number_definitions_ending_in_0_first_char = [item[:1] for item in list(two_number_definitions_ending_in_0['PSC CODE STRING'])]
    selector = (two_number_definitions['PSC CODE STRING'].str[1:2]=='1') & (~two_number_definitions['PSC CODE STRING'].str[:1].isin(two_number_definitions_ending_in_0_first_char))
    two_letter_definitions_ending_in_1 = two_number_definitions[selector]
    
    df_result = pd.concat([two_letter_definitions_ending_in_1, two_number_definitions_ending_in_0, single_letter_definitions]).sort_values(by='PSC CODE STRING').reset_index(drop=True) #two_number_definitions #single_letter_definitions
    df_result['PSC_Cat'] = df_result['PSC CODE STRING'].str[:1]
    return df_result

psc_cat_major = Retrieve_PSC_Main_Name()
psc_cat_major

,PSC CODE STRING,PRODUCT AND SERVICE CODE NAME,PSC_Cat
0,10,WEAPONS,1
1,20,SHIP AND MARINE EQUIPMENT,2
2,30,MECHANICAL POWER TRANSMISSION EQPT,3
3,40,"ROPE, CABLE, CHAIN, FITTINGS",4
4,51,HAND TOOLS,5
5,61,"ELECTRIC WIRE, POWER DISTRIB EQPT",6
6,7,IT AND TELECOM - INFORMATION TECHNOLOGY AND TE...,7
7,71,FURNITURE,7
8,80,"BRUSHES, PAINTS, SEALERS, ADHESIVES",8
9,91,"FUELS, LUBRICANTS, OILS, WAXES",9


### Compute and Sort $s by PSC

In [676]:
selector = (df_by_Dept_Agency_Office_PSC_Contractor['action_date_fiscal_year'] == '2019')
#selector = (df_share_by_PSC['PSC_Cat'] == 'B')
df_GFY19_PSC_totals = df_by_Dept_Agency_Office_PSC_Contractor[selector].groupby(['PSC_Cat','product_or_service_code', 'product_or_service_code_description'])['federal_action_obligation'].sum().reset_index()
df_GFY19_PSC_totals = df_GFY19_PSC_totals.merge(psc_cat_major, on=['PSC_Cat'])
df_GFY19_PSC_totals['PSC_Cat_TEXT'] = df_GFY19_PSC_totals['PSC_Cat'].apply(str) + "-" + df_GFY19_PSC_totals['PRODUCT AND SERVICE CODE NAME'].apply(str)
df_GFY19_PSC_totals

,PSC_Cat,product_or_service_code,product_or_service_code_description,federal_action_obligation,PSC CODE STRING,PRODUCT AND SERVICE CODE NAME,PSC_Cat_TEXT
0,1,1005,"GUNS, THROUGH 30MM",7.058200e+08,10,WEAPONS,1-WEAPONS
1,1,1010,"GUNS, OVER 30MM UP TO 75MM",5.613521e+07,10,WEAPONS,1-WEAPONS
2,1,1015,"GUNS, 75MM THROUGH 125MM",4.729029e+07,10,WEAPONS,1-WEAPONS
3,1,1020,"GUNS, OVER 125MM THROUGH 150MM",9.794340e+07,10,WEAPONS,1-WEAPONS
4,1,1025,"GUNS, OVER 150MM THROUGH 200MM",7.526742e+07,10,WEAPONS,1-WEAPONS
...,...,...,...,...,...,...,...
2677,Z,Z2PC,REPAIR OR ALTERATION OF UNIMPROVED REAL PROPER...,1.622039e+07,Z,"MAINT, REPAIR, ALTER REAL PROPERTY","Z-MAINT, REPAIR, ALTER REAL PROPERTY"
2678,Z,Z2PD,REPAIR OR ALTERATION OF WASTE TREATMENT AND ST...,2.618541e+07,Z,"MAINT, REPAIR, ALTER REAL PROPERTY","Z-MAINT, REPAIR, ALTER REAL PROPERTY"
2679,Z,Z2PZ,REPAIR OR ALTERATION OF OTHER NON-BUILDING FAC...,6.085614e+08,Z,"MAINT, REPAIR, ALTER REAL PROPERTY","Z-MAINT, REPAIR, ALTER REAL PROPERTY"
2680,Z,Z2QA,REPAIR OR ALTERATION OF RESTORATION OF REAL PR...,3.699807e+08,Z,"MAINT, REPAIR, ALTER REAL PROPERTY","Z-MAINT, REPAIR, ALTER REAL PROPERTY"


In [677]:
source = df_obligations_gfy[df_obligations_gfy['action_date_fiscal_year'] != '2020'] 

alt.Chart(source, title="Total Contractor Obligations GFY10-GFY19").mark_bar().encode(
    x=alt.X('action_date_fiscal_year'),
    y=alt.Y('federal_action_obligation', scale = alt.Scale(domain=[4e11,6e11])),
    tooltip=[alt.Tooltip('federal_action_obligation')]
).properties(
    width=400,
    height=300
)

alt.Chart(...)

In [803]:
selector = (df_combined['action_date_fiscal_year'] == '2019')
df_plot = df_combined[selector].sort_values(by=['federal_action_obligation'], ascending=False)


alt.Chart(df_plot).mark_bar().encode(
    x=alt.X('PSC_Cat:N',sort='-y'),
    y=alt.Y('federal_action_obligation:Q'),
    column='action_date_fiscal_year',
    color='PSC_Cat',
    order = alt.Order(
        'federal_action_obligation',
        sort='descending',
    )
).properties(
    width=600,
    height=500
)

alt.Chart(...)

In [680]:
sorted(df_GFY19_PSC_totals['PSC_Cat_TEXT'].unique())

['1-WEAPONS',
 '2-SHIP AND MARINE EQUIPMENT',
 '3-MECHANICAL POWER TRANSMISSION EQPT',
 '4-ROPE, CABLE, CHAIN, FITTINGS',
 '5-HAND TOOLS',
 '6-ELECTRIC WIRE, POWER DISTRIB EQPT',
 '7-FURNITURE',
 '7-IT AND TELECOM - INFORMATION TECHNOLOGY AND TELECOMMUNICATIONS',
 '8-BRUSHES, PAINTS, SEALERS, ADHESIVES',
 '9-FUELS, LUBRICANTS, OILS, WAXES',
 'A-RESEARCH AND DEVELOPMENT',
 'B-SPECIAL STUDIES/ANALYSIS, NOT R&D',
 'C-ARCHITECT/ENGINEER SERVICES',
 'D-IT AND TELECOM - INFORMATION TECHNOLOGY AND TELECOMMUNICATIONS',
 'E-PURCHASE OF STRUCTURES/FACILITIES',
 'F-NATURAL RESOURCES MANAGEMENT',
 'G-SOCIAL SERVICES',
 'H-QUALITY CONTROL, TEST, INSPECTION',
 'J-MAINT, REPAIR, REBUILD EQUIPMENT',
 'K-MODIFICATION OF EQUIPMENT',
 'L-TECHNICAL REPRESENTATIVE SVCS.',
 'M-OPERATION OF GOVT OWNED FACILITY',
 'N-INSTALLATION OF EQUIPMENT',
 'P-SALVAGE SERVICES',
 'Q-MEDICAL SERVICES',
 'R-SUPPORT SVCS (PROF, ADMIN, MGMT)',
 'S-UTILITIES AND HOUSEKEEPING',
 'T-PHOTO, MAP, PRINT, PUBLICATION',
 'U-EDUCATIO

## Sorted Curves

In [805]:
def Construct_Bar_Chart(df_share_by_PSC, psc_cat='D', gfy='2019', top_N_firms=10, y_scale_max = 0): #df = df_share_by_PSC

    selector = (df_share_by_PSC['action_date_fiscal_year'] == gfy) & (df_share_by_PSC['PSC_Cat'] == psc_cat)
    df_plot = df_share_by_PSC[selector].sort_values(by=['federal_action_obligation'], ascending=False)[:top_N_firms]
    obligations_total = df_share_by_PSC[selector]['federal_action_obligation'].sum()
    
    max_y_value = df_plot['federal_action_obligation'].max() * 1.1
    if y_scale_max == 0:
        y_scale_max = max_y_value
    

    altair_plot = alt.Chart(df_plot, title=[f"PSC_Cat: {psc_cat}"] + [f"Top N={top_N_firms} Companies"]).mark_bar().encode(
        x=alt.X('recipient_parent_name:N',sort='-y'),
        y=alt.Y('federal_action_obligation:Q', scale=alt.Scale(domain=[0, y_scale_max])),
        column='action_date_fiscal_year',
        color='recipient_parent_name',
        order = alt.Order(
            'federal_action_obligation',
            sort='descending',
        )
    ).properties(
        #title='My Title',
        width=300,
        height=300
    )
    
    return altair_plot, obligations_total, max_y_value

In [806]:
def Display_Altair_Bar_Charts(df_share_by_PSC, psc_cat = 'D', gfy_1 = '2019', gfy_2 = '2014'):
    altair_plot_1, obligations_total_1, y_scale_max = Construct_Bar_Chart(df_share_by_PSC, psc_cat = psc_cat, gfy = gfy_1, top_N_firms=10, y_scale_max = 0)
    altair_plot_2, obligations_total_2, y_scale_max = Construct_Bar_Chart(df_share_by_PSC, psc_cat = psc_cat, gfy = gfy_2, top_N_firms=10, y_scale_max=y_scale_max)

    print()
    print(f"PSC_Cat: {psc_cat}")
    print(f"GFY{gfy_1}/GFY{gfy_2}: {obligations_total_1/obligations_total_2:.3f}")
    print(f"GFY{gfy_1} Total Obligations: ${obligations_total_1/1e9:.3f}B, GFY{gfy_2}: ${obligations_total_2/1e9:.3f}B")
    print()

    return altair_plot_1 | altair_plot_2

In [807]:
df_share_by_PSC = df_by_Dept_Agency_Office_PSC_Contractor.groupby(['action_date_fiscal_year','PSC_Cat','recipient_parent_name'])['federal_action_obligation'].sum().reset_index()
df_share_by_PSC.head()

,action_date_fiscal_year,PSC_Cat,recipient_parent_name,federal_action_obligation
0,2010,1,126536 CANADA INC,70080.0
1,2010,1,17TH ST. PHOTO SUPPLY INC.,11564.9
2,2010,1,1DB - METRAVIB,27970.0
3,2010,1,200 PARK INC.,20250.0
4,2010,1,2037706 ONTARIO INC,11909.0


In [808]:
Display_Altair_Bar_Charts(df_share_by_PSC, psc_cat = 'R', gfy_1 = '2019', gfy_2 = '2014')


PSC_Cat: R
GFY2019/GFY2014: 1.315
GFY2019 Total Obligations: $85.911B, GFY2014: $65.333B



alt.HConcatChart(...)

In [809]:
Display_Altair_Bar_Charts(df_share_by_PSC,psc_cat = 'D', gfy_1 = '2019', gfy_2 = '2014')


PSC_Cat: D
GFY2019/GFY2014: 1.397
GFY2019 Total Obligations: $44.171B, GFY2014: $31.619B



alt.HConcatChart(...)

In [818]:
# sort and get the 10 largest every GFY
df_share_by_PSC = df_by_Dept_Agency_Office_PSC_Contractor.groupby(['action_date_fiscal_year','PSC_Cat','recipient_parent_name'])['federal_action_obligation'].sum().reset_index()
df_share_by_PSC.head()

psc_cat = 'D'
selector = (df_share_by_PSC['PSC_Cat'] == psc_cat) & (df_share_by_PSC['action_date_fiscal_year'] != '2020')
#!!!! this is critical group_keys=False
top_N = 10
df_plot = df_share_by_PSC[selector].groupby("action_date_fiscal_year", group_keys=False).apply(lambda xgroup: xgroup.sort_values(by=['federal_action_obligation'], ascending=[False]).head(top_N))

df_plot

# Streamgraph https://altair-viz.github.io/gallery/streamgraph.html

source = df_plot

selection = alt.selection_multi(fields=['recipient_parent_name'], bind='legend')

chart = alt.Chart(source, title=f'Evolution of top {top_N} Contractors for PSC_Cat: {psc_cat}').mark_area().encode(
    alt.X('action_date_fiscal_year',
        axis=alt.Axis(domain=False, tickSize=0)
    ),
    alt.Y('federal_action_obligation', stack='center', axis=None),
    alt.Color('recipient_parent_name:N',
        scale=alt.Scale(scheme='category20b'),
              
    ),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)), # this is what adjust the highlighting when you select something
).properties(
    width=600,
    height=400
).add_selection(
    selection
) #.interactive()

# chart.save('chart.png')
# chart.save('chart.svg')
# chart.save('chart.pdf')
# chart.save('chart.html')

chart

alt.Chart(...)

In [820]:
# Explore mix of business for the top 10 in D
gfy = '2019'
psc_cat = 'D'
top_N_firms = 10
selector = (df_share_by_PSC['action_date_fiscal_year'] == gfy) & (df_share_by_PSC['PSC_Cat'] == psc_cat)
df_plot = df_share_by_PSC[selector].sort_values(by=['federal_action_obligation'], ascending=False)[:top_N_firms]

top_N_parent_names = sorted(df_plot['recipient_parent_name'].unique())

selector = (df_by_Dept_Agency_Office_PSC_Contractor['action_date_fiscal_year']==gfy) & (df_by_Dept_Agency_Office_PSC_Contractor['recipient_parent_name'].isin(top_N_parent_names))
df_explore = df_by_Dept_Agency_Office_PSC_Contractor[selector].groupby(['recipient_parent_name', 'PSC_Cat'])['federal_action_obligation'].sum().reset_index()
df_explore_pivot = df_explore.query('federal_action_obligation > 1e6').pivot_table(index=['recipient_parent_name'], columns=['PSC_Cat'], values=['federal_action_obligation'], aggfunc=np.sum, margins=True, fill_value=0.0)
df_explore_pivot

df_explore_all = df_explore.groupby(['PSC_Cat'])['federal_action_obligation'].sum().reset_index()
df_explore_all['recipient_parent_name'] = "Top_N_Subset_Total"
df_explore_all

source = pd.concat([df_explore, df_explore_all]).query("federal_action_obligation > 1e6")

alt.Chart(source, title=f"PSC_Cat: {psc_cat} - Top {top_N_firms} Contractors and their Full Mix of Business in other PSC Categories").mark_point().encode(
    x=alt.X('PSC_Cat'),
    y=alt.Y('federal_action_obligation'),
    size = alt.Size('federal_action_obligation:Q'),
    color = alt.Color('recipient_parent_name:N'),
    tooltip = [alt.Tooltip('recipient_parent_name')]
).properties(
width=600,
height=400
)

alt.Chart(...)

In [774]:
#compute total prime obligations from this top 10
# total $'s and as fraction of FY19 Obs
df_explore_all['federal_action_obligation'].sum()/1e9, df_explore_all['federal_action_obligation'].sum()/df_obligations_gfy[df_obligations_gfy['action_date_fiscal_year']==gfy]['federal_action_obligation'].sum()

(58.75897533372999, 0.0996086692542524)

## Explore Market Share

In [697]:
selection = alt.selection_multi(fields=['PSC_Cat'], bind='legend', on='mouseover', nearest=True)

y_field = 'Top_1_Marketshare'

#interpolate='linear'
selector = (df_combined['action_date_fiscal_year']!='2020')# & (df_combined['PSC_Cat'].isin(['1','A','B','D','R']))
source = df_combined[selector]
chart = alt.Chart(source, title='PSC_Cats - ' + y_field).mark_line(interpolate='basis').encode(
    x='action_date_fiscal_year',
    y=y_field,
    color=alt.Color('PSC_Cat:N',legend=alt.Legend(symbolLimit=50)),
    tooltip = ['PSC_Cat:N',"Top_1_Marketshare"],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)), # this is what adjust the highlighting when you select something
).properties(
height=600,
width=600).add_selection(
    selection,
    #highlight
) #.interactive()

chart

alt.Chart(...)

In [695]:
selection = alt.selection_multi(fields=['PSC_Cat'], bind='legend', on='mouseover', nearest=True)

y_field = 'Top_1_Marketshare'

#interpolate='linear'
selector = (df_combined['action_date_fiscal_year']!='2020') & (df_combined['PSC_Cat'].isin(['A','B','D','R','Q','M','S']))
source = df_combined[selector]
chart = alt.Chart(source, title='Services Focused PSC_Cats - ' + y_field).mark_line(interpolate='basis').encode(
    x='action_date_fiscal_year',
    y=y_field,
    color=alt.Color('PSC_Cat:N',legend=alt.Legend(symbolLimit=50)),
    tooltip = ['PSC_Cat:N',"Top_1_Marketshare"],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)), # this is what adjust the highlighting when you select something
).properties(
height=600,
width=600).add_selection(
    selection,
    #highlight
) #.interactive()

chart

alt.Chart(...)

In [778]:
selection = alt.selection_multi(fields=['PSC_Cat'], bind='legend', on='mouseover', nearest=True)

y_field = 'Top_5_Marketshare'

#interpolate='linear'
selector = (df_combined['action_date_fiscal_year']!='2020') & (df_combined['PSC_Cat'].isin(['A','B','D','R','Q','M','S']))
source = df_combined[selector]
chart = alt.Chart(source, title='Services Focused PSC_Cats - ' + y_field).mark_line(interpolate='basis').encode(
    x='action_date_fiscal_year',
    y=y_field,
    color=alt.Color('PSC_Cat:N',legend=alt.Legend(symbolLimit=50)),
    tooltip = ['PSC_Cat:N',"Top_1_Marketshare"],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)), # this is what adjust the highlighting when you select something
).properties(
height=600,
width=600).add_selection(
    selection,
    #highlight
) #.interactive()

chart

alt.Chart(...)

In [821]:
selection = alt.selection_multi(fields=['PSC_Cat'], bind='legend', on='mouseover', nearest=True)

y_field = 'Top_1_Marketshare'

#interpolate='linear'
selector = (df_combined['action_date_fiscal_year']!='2020') & (df_combined['PSC_Cat'].isin(['A','B','D','R','Q','M','S']))
source = df_combined[selector]
chart = alt.Chart(source, title='Services Focused PSC_Cats - ' + y_field).mark_circle().encode(
    x='federal_action_obligation',
    y=y_field,
    color=alt.Color('PSC_Cat:N',legend=alt.Legend(symbolLimit=50)),
    tooltip = ['PSC_Cat:N',"Top_1_Marketshare"],
    #opacity=alt.condition(selection, alt.value(1), alt.value(0.2)), # this is what adjust the highlighting when you select something
).properties(
height=400,
width=500
)#.add_selection(selection,#) #.interactive()

chart

alt.Chart(...)

In [741]:
# create reusable plotting function to use at Dept, Agency, Office elevel

def Plot_TopN_Market_Share_Trends(df_combined, plot_field = "Top_1_Marketshare" , psc_cat_list = ['D','R','A'], gfy_min = 2010, gfy_max=2019):
    '''Reusable plotting function
    
    df_combined fields: 
    ['action_date_fiscal_year', 'action_date_fiscal_year_numeric', 'PSC_Cat',
       'federal_action_obligation', 'Top_1_Companies_Obligations',
       'Top_2_Companies_Obligations', 'Top_5_Companies_Obligations',
       'Top_1_Marketshare', 'Top_2_Marketshare', 'Top_5_Marketshare']
    '''
    
    # check if there is only 1 department and 1 agency
    if 'funding_agency_name' in df_combined.columns:
        departments = list(df_combined['funding_agency_name'].unique())
    else:
        departments =['ALL Departments']
    if 'funding_sub_agency_name' in df_combined.columns:
        agencies = list(df_combined['funding_sub_agency_name'].unique())
    else:
        agencies = ['ALL Agencies']

    selector = (df_combined['PSC_Cat'].isin(psc_cat_list))

    title = [plot_field + "-" + departments[0] + "-"] +[ "\n" + agencies[0]]
        
    y_field = plot_field + ':Q'

    source = df_combined[selector]

    plot_1 = alt.Chart(source, title=title).mark_circle(opacity=0.75).encode(
        alt.X('federal_action_obligation:Q'),
        alt.Y(y_field),
        alt.Color('PSC_Cat:N'),
        tooltip = [alt.Tooltip('action_date_fiscal_year_numeric'), alt.Tooltip('PSC_Cat')]
    )
    
#     title = "Top_1_Marketshare"
#     y_field = title + ':Q'

#     select_year = alt.selection_single(
#         name='Select', fields=['action_date_fiscal_year_numeric'], init={'action_date_fiscal_year_numeric': gfy_min},
#         bind=alt.binding_range(min=gfy_min, max=gfy_max, step=1)
#     )

#     plot_2 = alt.Chart(source, title=title).mark_circle(opacity=0.75).encode(
#         alt.X('federal_action_obligation:Q'),
#         alt.Y(y_field),
#         alt.Color('PSC_Cat:N'),
#         tooltip = [alt.Tooltip('action_date_fiscal_year_numeric'), alt.Tooltip('PSC_Cat')],
#     ).add_selection(select_year).transform_filter(select_year)

#     #plot_2 | plot_1
#     #plot_2


    #source = df_combined[selector]
    plot_3 = alt.Chart(source, title=title).mark_circle(opacity=0.75).encode(
        alt.X('action_date_fiscal_year:N'),
        alt.Y(y_field),
        alt.Size('federal_action_obligation:Q'),
        alt.Color('PSC_Cat:N'),
        tooltip = [alt.Tooltip('action_date_fiscal_year_numeric'), alt.Tooltip('PSC_Cat')]
    ).properties(
        width=400,
        height=300
    )


    return plot_1 | plot_3

In [717]:
df_combined = Create_df_combined(df_by_Dept_Agency_Office_PSC_Contractor, by_group=['PSC_Cat'])

In [742]:
Plot_TopN_Market_Share_Trends(df_combined, plot_field = "Top_1_Marketshare", psc_cat_list = ['D','R','A'], gfy_min = 2010, gfy_max=2019)

alt.HConcatChart(...)

In [743]:
Plot_TopN_Market_Share_Trends(df_combined, plot_field = "Top_5_Marketshare", psc_cat_list = ['D','R','A'], gfy_min = 2010, gfy_max=2019)

alt.HConcatChart(...)

In [572]:
# Revise to only sort through FY19 via read CSV files
funding_agency_names = sorted(df_by_Dept_Agency_Office_PSC_Contractor['funding_agency_name'].unique())
funding_agency_names

['ACCESS BOARD (USAB)',
 'ADMINISTRATIVE CONFERENCE OF THE U.S. (ACUS)',
 'ADVISORY COUNCIL ON HISTORIC PRESERVATION (ACHP)',
 'AFRICAN DEVELOPMENT FOUNDATION (EOP)',
 'AGENCY FOR INTERNATIONAL DEVELOPMENT (USAID)',
 'AMERICAN BATTLE MONUMENTS COMMISSION (ABMC)',
 'ARCHITECT OF THE CAPITOL (AOC)',
 'ARMED FORCES RETIREMENT HOME (AFRH)',
 'BROADCASTING BOARD OF GOVERNORS (BBG)',
 'CENTRAL INTELLIGENCE AGENCY (CIA)',
 'COMMISSION OF FINE ARTS (CFA)',
 'COMMISSION ON CIVIL RIGHTS (USCCR)',
 'COMMITTEE FOR PURCHASE FROM PEOPLE WHO ARE BLIND OR SEVERELY DISABLED (ABILITYONE)',
 'COMMODITY FUTURES TRADING COMMISSION (CFTC)',
 'CONSUMER FINANCIAL PROTECTION BUREAU (CFPB)',
 'CONSUMER PRODUCT SAFETY COMMISSION (CPSC)',
 'CORPORATION FOR NATIONAL AND COMMUNITY SERVICE (CNCS)',
 'CORPS OF ENGINEERS - CIVIL WORKS (USACE)',
 'COURT SERVICES AND OFFENDER SUPERVISION AGENCY (CSOSA)',
 'DEFENSE NUCLEAR FACILITIES SAFETY BOARD (DNFSB)',
 'DENALI COMMISSION (DENALI)',
 'DEPARTMENT OF AGRICULTURE (USDA)

### Look at Department-Level Patterns

In [704]:
%%time
df_combined_Department_PSC_Cat = Create_df_combined(df_by_Dept_Agency_Office_PSC_Contractor, by_group=['funding_agency_name','PSC_Cat'])
# TEST CHECK
assert GFY_Obligation_Totals_Checksum_TEST(df_by_Dept_Agency_Office_PSC_Contractor, df_combined_Department_PSC_Cat)

CPU times: user 32.6 s, sys: 2.15 s, total: 34.7 s
Wall time: 33.5 s


True

#### Examine DoD

In [784]:
selector = (df_combined_Department_PSC_Cat['action_date_fiscal_year']!='2020') # at the time of this analysis GFY20 was not fully reported
df_combined_subset = df_combined_Department_PSC_Cat[df_combined_Department_PSC_Cat['funding_agency_name']=='DEPARTMENT OF DEFENSE (DOD)']

Plot_TopN_Market_Share_Trends(df_combined_subset, plot_field = "Top_1_Marketshare", psc_cat_list = ['D','R','A'], gfy_min = 2010, gfy_max=2019)

alt.HConcatChart(...)

In [785]:
Plot_TopN_Market_Share_Trends(df_combined_subset, plot_field = "Top_5_Marketshare", psc_cat_list = ['D','R','A'], gfy_min = 2010, gfy_max=2019)

alt.HConcatChart(...)

#### Example DoD - DISA

In [786]:
%%time
selector = (df_by_Dept_Agency_Office_PSC_Contractor['action_date_fiscal_year']!='2020') #at the time of this analysis - GFY20 is not 100% complete
df_combined_Department_AGENCY_PSC_Cat = Create_df_combined(df_by_Dept_Agency_Office_PSC_Contractor[selector], by_group=['funding_agency_name','funding_sub_agency_name','PSC_Cat'])
# TEST CHECK
assert GFY_Obligation_Totals_Checksum_TEST(df_by_Dept_Agency_Office_PSC_Contractor, df_combined_Department_PSC_Cat)

CPU times: user 1min 14s, sys: 4.87 s, total: 1min 19s
Wall time: 1min 16s


In [787]:
# Revise to only sort through FY19 via read CSV files
selector = df_combined_Department_AGENCY_PSC_Cat['funding_agency_name']=='DEPARTMENT OF DEFENSE (DOD)'
funding_sub_agency_names = sorted(df_combined_Department_AGENCY_PSC_Cat[selector]['funding_sub_agency_name'].unique())
funding_sub_agency_names

['AIR COMBAT COMMAND',
 'AIR FORCE ELEMENTS, U.S. CENTRAL COMMAND',
 'AIR NATIONAL GUARD',
 'AIR NATIONAL GUARD UNITS (TITLE 32)',
 'ARMY CORPS OF ENGINEERS CIVIL WORKS',
 'BUSINESS TRANSFORMATION AGENCY',
 'COUNTERINTELLIGENCE FIELD ACTIVITY (CIFA)',
 'DEFENSE ADVANCED RESEARCH PROJECTS AGENCY  (DARPA)',
 'DEFENSE COMMISSARY AGENCY  (DECA)',
 'DEFENSE CONTRACT AUDIT AGENCY',
 'DEFENSE CONTRACT MANAGEMENT AGENCY (DCMA)',
 'DEFENSE COUNTERINTELLIGENCE AND SECURITY AGENCY',
 'DEFENSE FINANCE AND ACCOUNTING SERVICE (DFAS)',
 'DEFENSE HEALTH AGENCY (DHA)',
 'DEFENSE HUMAN RESOURCES ACTIVITY',
 'DEFENSE INFORMATION SYSTEMS AGENCY (DISA)',
 'DEFENSE INTELLIGENCE AGENCY',
 'DEFENSE LOGISTICS AGENCY',
 'DEFENSE MEDIA ACTIVITY (DMA)',
 'DEFENSE MICROELECTRONICS ACTIVITY (DMEA)',
 'DEFENSE POW/MIA OFFICE',
 'DEFENSE SECURITY COOPERATION AGENCY',
 'DEFENSE THREAT REDUCTION AGENCY (DTRA)',
 'DEPT OF DEFENSE',
 'DEPT OF DEFENSE EDUCATION ACTIVITY (DODEA)',
 'DEPT OF THE AIR FORCE',
 'DEPT OF THE AR

In [822]:
selector = (df_combined_Department_AGENCY_PSC_Cat['funding_agency_name']=='DEPARTMENT OF DEFENSE (DOD)') & (df_combined_Department_AGENCY_PSC_Cat['funding_sub_agency_name']=='DEFENSE INFORMATION SYSTEMS AGENCY (DISA)')
df_combined_subset = df_combined_Department_AGENCY_PSC_Cat[selector]

In [823]:
Plot_TopN_Market_Share_Trends(df_combined_subset, plot_field = "Top_1_Marketshare", psc_cat_list = ['D','R'], gfy_min = 2010, gfy_max=2019)

alt.HConcatChart(...)

In [824]:
Plot_TopN_Market_Share_Trends(df_combined_subset, plot_field = "Top_5_Marketshare", psc_cat_list = ['D','R'], gfy_min = 2010, gfy_max=2019)

alt.HConcatChart(...)

#### Examine SSA

In [825]:
df_combined_subset = df_combined_Department_PSC_Cat[df_combined_Department_PSC_Cat['funding_agency_name']=='SOCIAL SECURITY ADMINISTRATION (SSA)']

In [826]:
Plot_TopN_Market_Share_Trends(df_combined_subset, plot_field = "Top_1_Marketshare", psc_cat_list = ['D','R'], gfy_min = 2010, gfy_max=2019)

alt.HConcatChart(...)

In [827]:
Plot_TopN_Market_Share_Trends(df_combined_subset, plot_field = "Top_5_Marketshare", psc_cat_list = ['D','R'], gfy_min = 2010, gfy_max=2019)

alt.HConcatChart(...)

### END

<!-- https://opensource.org/licenses/MIT
MIT Open Source License Copyright 2020 Leif C Ulstrup
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE. -->

In [796]:
# https://opensource.org/licenses/MIT
# MIT Open Source License Copyright 2020 Leif C Ulstrup
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
# The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.